In [1]:
print('Analysis Dashbdddddoard')

Analysis Dashboard


In [2]:
from ipywidgets import interact, interactive, widgets, interact_manual
import glob
import pandas as pd
import plotly.graph_objects as go

from scipy.signal import savgol_filter
import peakutils

from ipyfilechooser import FileChooser
%matplotlib widget

In [3]:
def plot(thresh, peakdist, buffer, polynomial, window):
    peaks = []
    basepoints = []
    frontpoints = []
    
    for i in range(0, len(vid.origin)):
        vid.dataframe[i]['disp'] = savgol_filter(vid.origin[i]['disp'], window, polynomial)
        vid.dataframe[i]['disp'] = vid.dataframe[i]['disp'] * -1
        peaks.append(peakutils.indexes(vid.dataframe[i]['disp'], thresh, peakdist))
        peaks[i] = peaks[i][1:-1]
        basepoints.append([])
        frontpoints.append([])
        for peak in peaks[i]:
            for k in range(peak - buffer, 1, -1):
                dfdt = (vid.dataframe[i]['disp'][k] - vid.dataframe[i]['disp'][k - 1])
                if dfdt<=0:
                    basepoints[i].append(k)
                    break
            for k in range(peak + buffer, 1000, 1):
                dfdt = (vid.dataframe[i]['disp'][k + 1] - vid.dataframe[i]['disp'][k])
                if dfdt>=0:
                    frontpoints[i].append(k)
                    break
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=vid.dataframe[i]['time'],y=vid.dataframe[i]['disp'], name = 'Smooth Data')) 
        fig.add_trace(go.Scatter(x=vid.dataframe[i]['time'][peaks[i]], y=vid.dataframe[i]['disp'][peaks[i]], mode = 'markers', name = 'Peaks'))
        fig.add_trace(go.Scatter(x=vid.dataframe[i]['time'][basepoints[i]], y=vid.dataframe[i]['disp'][basepoints[i]], mode = 'markers', name = 'Basepoints'))
        fig.add_trace(go.Scatter(x=vid.dataframe[i]['time'][frontpoints[i]], y=vid.dataframe[i]['disp'][frontpoints[i]], mode = 'markers', name = 'Frontpoints'))
        fig.show()


In [4]:
class fileselection():
    def __init__(self):
        self.fc = FileChooser('/srv/scratch/Data/tracking_generated/')
        display(self.fc)

        self.fc.use_dir_icons = True
        self.fc.title = '<b>Choose Folder</b>'
        self.fc.register_callback(self.folderselected)
        
    def folderselected(self, chooser):
        self.dataframe = []
        self.origin = []
        files = glob.glob(self.fc.selected_path + '/*')
        for i in range(len(files)):
            self.dataframe.append([])
            self.origin.append([])
            self.dataframe[i] =  pd.read_csv(files[i])
            self.origin[i] = pd.read_csv(files[i])
            print(i)
        print(len(vid.origin))
        #self.paramaterization(.6, 10, 5, 3, 13)
vid = fileselection()


FileChooser(path='/srv/scratch/Data/tracking_generated', filename='', show_hidden='False')

0
1
2


In [6]:
@interact(thres = (0, 1, .1), peakdist = (0, 30), buffer = (0, 20), poly = (1, 16), window = (9, 31, 2))
def paramaterization(thres = .6, peakdist = 10, buffer = 5, poly = 3, window = 13):
    if vid.fc.selected_path is not None:
        plot(thres, peakdist, buffer, poly, window)
        

interactive(children=(FloatSlider(value=0.6, description='thres', max=1.0), IntSlider(value=10, description='p…